# Проектная работа: Иccледование рынка заведений общественного питания Москвы

## Описание проекта

Инвесторы из фонда «Shut Up and Take My Money» решили попробовать себя в новой области и открыть заведение общественного питания в Москве. Заказчики ещё не знают, что это будет за место: кафе, ресторан, пиццерия, паб или бар, — и какими будут расположение, меню и цены.
Необходимо подготовить исследование рынка Москвы, найти интересные особенности и презентовать полученные результаты, которые в будущем помогут в выборе подходящего инвесторам места.

Доступен датасет с заведениями общественного питания Москвы, составленный на основе данных сервисов Яндекс Карты и Яндекс Бизнес на лето 2022 года. Информация, размещённая в сервисе Яндекс Бизнес, могла быть добавлена пользователями или найдена в общедоступных источниках. Она носит исключительно справочный характер.

В файле `moscow_places.csv` содержится различная информация о заведениях: название, часы работы, средний чек, рейтинг, административный район, в котором находится заведение и т.д.

## Оглавление проекта

1. [Открытие данных](#start)
2. [Предобработка данных](#continue)
    * [Обработка пропущенных значений](#lost)
    * [Обработка дубликатов](#duplicates)
    * [Добавление столбцов](#rows)
3. [Анализ данных](#middle)
    * [Исследование категорий объектов общественного питания](#categories)
    * [Исследование количества посадочных мест в местах по категориям](#quantity)
    * [Исследование соотношения сетевых и несетевых заведений](#chain)
    * [Исследование соотношения количества сетевых и несетевых заведений по категориям объектов](#chain_2)
    * [Исследование топ-15 популярных сетей в Москве](#top15chains)
    * [Изучение общего количества заведений и количества заведений каждой категории по административным районам Москвы](#quantity_moscow)
    * [Распределение средних рейтингов по категориям заведений](#rating)
    * [Фоновая картограмма (хороплет) со средним рейтингом заведений каждого района](#map)
    * [Исследование топ-15 улиц по количеству заведений](#top15streets)
    * [Изучение значений средних чеков заведений по округам Москвы](#middle_check)
    * [Исследование зависимости рейтинга и среднего чека в заведениях](#rate_check)
4. [Детализация исследования: открытие кофейни](#final)
    * [Исследование количества и расположения кофеен в датасете, а также рейтинга кофеен по районам](#cafesquantity)
    * [Исследование наличия круглосуточных кофеен](#cafes24_7)
    * [Исследование стоимости чашки капучино в кофейнях по районам](#cap_sum)

### Загрузка данных и изучение общей информации <a class="anchor" id="start"></a>

#### 1. Загрузка библиотек 

In [5]:
import pandas as pd
#импортируем все библиотеки, которые могут понадобиться для построения графиков
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import Marker, Map
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import plotly.io as pio
pio.renderers.default = "svg"
svg_renderer = pio.renderers["svg"]
svg_renderer.scale = 1.2

ModuleNotFoundError: No module named 'folium'

#### 2. Изучение общей информации

In [ ]:
try: 
    df = pd.read_csv('/datasets/moscow_places.csv')
except:
    df = pd.read_csv('moscow_places.csv')

In [ ]:
#изучим информацию о данных
df.info()

In [ ]:
#изучим первые 20 строк данных
df.head(20)

В датасете представлено 8406 заведений. 

В столбцах `hours`, `price`, `avg_bill`, `middle_avg_bill`, `middle_coffee_cup`, `seats` есть пропущенные значения. Заполнить пропущенные значения в столбцах `hours` и `seats` не представляется возможным, так как мы не можем проверить часы работы заведения и количество посадочных мест, если данные не были указаны владельцем заведения или кем-то из пользователей. 
Средняя стоимость заказа также может быть не указана, если заведение, например, новое. Также можно обратить внимание, что зачастую значения отсутствуют одновременно в трех столбцах `price`, `avg_bill`, `middle_avg_bill`. 

### Предобработка данных <a id="continue"></a>

#### 3. Проверка пропущенных значений  <a id="lost"></a>

In [ ]:
#проверяем пропущенные значения
df.isna().sum()

In [ ]:
#посмотрим на процентное соотношение пропусков
pd.DataFrame(round(df.isna().mean().sort_values(ascending=False)*100,1)).style.background_gradient('coolwarm')

#### 4. Проверка дубликатов в датасете<a id="duplicates"></a>

In [ ]:
#проверяем скрытые полные дубликаты
print('Количество скрытых полных дубликатов в датасете:', df.duplicated().sum())

In [ ]:
#приведем названия заведений в столбце к нижнему регистру, т.к. одно название может быть написано по-разному
df['name'] = df['name'].str.lower()

In [ ]:
#проверим все категории, которые есть в датасете
df['category'].unique()

In [ ]:
#исключим из наименований категорию заведения
sort = ['кафе ', 'ресторан ', 'столовая ',
             'булочная', 'кофейня ', 'пиццерия ',
             'быстрое питание ', 'бар, паб ']
for i in sort:
    df['name'] = df['name'].str.replace(i, '')

In [ ]:
#проверим дубликаты по 2 столбцам - названию заведения и адресу
df.duplicated(subset=['name', 'address']).sum()

In [ ]:
#удаляем дубликаты
df = df.drop_duplicates(['name', 'address'])

In [ ]:
#проверяем удаление дубликатов
df.duplicated(subset=['name', 'address']).sum()

Пропуски оставляем как есть, так как заполнить их средними значениями невозможно, либо мы не обладаем какими-то данными (выше упомянули посадочные места и время работы).

#### 5. Создаем столбец `street` с названиями улиц из столбца с адресом <a id="rows"></a>

In [ ]:
#применим метод split
df['street'] = df['address'].str.split(', ').str[1]

In [ ]:
#обработать корректно весь массив не выходит, поэтому для будущего исследования запомним, что не для каждого заведения улица указана корректно
df['street'].head(30)

#### 6. Создаем столбец `is_24/7` с обозначением, что заведение работает ежедневно и круглосуточно (24/7)
логическое значение True — если заведение работает ежедневно и круглосуточно; логическое значение False — в противоположном случае.

In [ ]:
#создадим функцию для заполнения столбца данными
def hours_24(value):
    if value['hours'] == 'ежедневно, круглосуточно':
        return 'True'
    if value['hours'] == 'Неизвестно':
        return 'Неизвестно'
    else:
        return 'False'

In [ ]:
df['is_24/7'] = df.apply(hours_24, axis=1)

**Промежуточный вывод по результатам предобработки данных:**
- всего в изначальном датасете у нас было 8406 мест, после предобработки данных осталось 8400 мест;
- в столбцах `hours`, `price`, `avg_bill`, `middle_avg_bill`, `middle_coffee_cup`, `seats` есть пропущенные значения, однако не заполняем пропуски и оставляем их как есть, пропущенные значения могли появиться, т.к. мало данных о заведении (оно новое, либо никто еще не успел указать данные в открытых источниках), либо, если владелец сам не указал информацию о заведении;
- дополнили таблицу 2 столбцами - `street` с названием улицы, на которой расположено заведение, и `is_24/7`, в котором могут содержаться три вида значений: "ежедневно, круглосуточно", если заведение работает 24/7, если в столбце `hours` указано "неизвестно", то аналогичное значение будет и в столбце `is_24/7`, а если в столбце `hours` отсутствуют сведения, то в столбец `is_24/7` подтянется False.

### Анализ данных <a id="middle"></a>

#### 7. Исследование категорий объектов общественного питания <a id="categories"></a>

In [ ]:
# готовим данные для графика
categories = pd.DataFrame(df['category'].value_counts()).reset_index()
# строим диаграмму с сегментами
fig = go.Figure(data=[go.Pie(labels=categories['index'], # указываем значения, которые появятся на метках сегментов
                             values=categories['category'], # указываем данные, которые отобразятся на графике
                             pull = [0.1, 0])]) # добавляем аргумент, который выделит сегмент-лидер на графике
fig.update_layout(title='Количество объектов общественного питания по категориям', # указываем заголовок графика
                  width=800, # указываем размеры графика
                  height=600,
                  annotations=[dict(x=1.12, # вручную настраиваем аннотацию легенды
                                    y=1.05,
                                    text='Категории объектов',
                                    showarrow=False)])
fig.show() # выводим график

In [ ]:
fig = px.bar(categories, 
             x='index', 
             y='category', 
             text='category',
             labels='category')
fig.update_layout(title='Количество заведений по категориям',
                  yaxis_title='Количество заведений',
                  xaxis_title='Категория заведений')
fig.show()

Больше всего в датасете заведений в категории "кафе" - 2377 заведений, а именно 28.3% от общего числа заведений, затем идут рестораны - 2039 заведений, а именно 24.3% от общего числа заведений, затем кофейни - 1413 заведений, т.е. 16.8% от общего числа заведений, затем 764 бара/паба, затем пиццерии - 633 заведения, заведения быстрого питания - 603, столовая - 315, булочная - 256. 

#### 8. Исследование количества посадочных мест в местах по категориям: рестораны, кофейни, пиццерии, бары и т.д. <a id="quantity"></a>

In [ ]:
#посмотрим в общем на распределение значений, применив describe()
df['seats'].describe()

Видим, что заведения бывают ну очень большими - 1288 посадочных мест, но в среднем количество посадочных мест - 108.

In [ ]:
#создадим переменную seats
#группируем по категориям, применим median()
seats = df.groupby('category').agg({'seats': 'median'}).sort_values('seats', ascending=False)
seats

In [ ]:
#строим график
plt.figure(figsize=(15, 8))
sns.set_style('darkgrid')
#применим boxplot для построения графика, чтобы оценить распределения
sns.boxplot(x='category', y='seats', data=df, hue_order='seats')
plt.title('Распределение количества посадочных мест в зависимости от категории заведения')
plt.xlabel('Категория')
plt.ylabel('Количество посадочных мест')
plt.show()

Из графика видно, что меньше всего посадочных мест в заведениях со следующими категориями: пиццерия, булочная, кафе. Больше всего посадочных мест в ресторанах, барах/пабах и кофейнях. При этом в кафе, ресторанах и барах часто очень много посадочных мест - больше медианного значения.

#### 9. Исследование соотношения сетевых и несетевых заведений в датасете <a id="chain"></a>

In [ ]:
#посмотрим в целом на количество сетевых заведений и заведений, которые не являются сетевыми
net = df.groupby('chain').agg({'name': 'count'}).rename(columns={'name': 'count'}).sort_values('count', ascending=False)
chain = ['Несетевые заведения', 'Сетевые заведения']
values = [5197, 3203]

In [ ]:
#изобразим распределение заведений на графике
fig = go.Figure(data=[go.Pie(labels=chain, values=values)])
fig.show()

Сетевых заведений - 38.1% от всех заведений, а несетевые заведения составляют 61.9% от всего датасета. 

#### 10. Какие категории заведений чаще являются сетевыми? <a id="chain_2"></a>

In [ ]:
chain = df.groupby('category').agg({'chain':['count','sum']}).reset_index()
# считаем количество заведений по категориям и сколько из них сетевые

chain.columns = ['category','total_amnt','chain']
# переименовывем столбцы

chain['percent%'] = (chain['chain']/chain['total_amnt']*100).round(1)
# считаем долю сетевых от всех заведений в категориях, выводим на экран

chain = chain.sort_values(by='percent%',ascending=False)
display(chain.style.background_gradient(subset=['total_amnt','percent%'],cmap='Blues', axis=0))

#для сортировки графика
order_list = df.groupby(['category'])['chain'].mean().sort_values(ascending=True).index

In [ ]:
fig = px.bar(chain,
             x='total_amnt',
             y='category',
             text= 'percent%',
             color='percent%',
             category_orders={"chain":['сетевое', 'несетевое']},
             height=500,
             width=700
             )
fig.update_layout(title = 'График, отражающий количество сетевых заведений в каждой категории',
                  xaxis_title = 'Общее количество заведений',
                  yaxis_title = 'Категории',
                  yaxis={'categoryarray':order_list}
                  )
fig.show()

In [ ]:
fig = px.bar(chain,
             x='total_amnt',
             y='category',
             text= 'percent%',
             color='percent%',
             category_orders={"chain":['сетевое', 'несетевое']},
             height=500,
             width=700
             )
fig.update_layout(title = 'График, отражающий количество сетевых заведений в каждой категории',
                  xaxis_title = 'Общее количество заведений',
                  yaxis_title = 'Категории',
                  yaxis={'categoryorder':'total ascending'}
                  )
fig.show()

Исходя из графика можем сделать вывод, что больше всего сетевых заведений в категориях булочная, пиццерия, кофейня. В булочных сетевых заведений свыше 61.3%, в пиццериях - 52.1%, в кофейнях - 51%, а вот меньше всего сетевых заведений в категории бар,паб - всего 22%.

In [ ]:
chain = df.groupby('category').agg({'chain':['count','sum']}).reset_index()
# считаем количество заведений по категориям и сколько из них сетевые

chain.columns = ['category','total_amnt','chain']
# переименовывем столбцы

chain['percent%'] = (chain['chain']/chain['total_amnt']*100).round(1)
# считаем долю сетевых от всех заведений в категориях, выводим на экран

chain = chain.sort_values(by='percent%',ascending=False)
display(chain.style.background_gradient(subset=['total_amnt','percent%'],cmap='Blues', axis=0))

order = list(chain['category'])
# список для сортировки графика

#### 11. Исследование топ-15 популярных сетей в Москве <a id="top15chains"></a>

In [ ]:
#создадим переменную top_15_chain
#сделаем срез по сетевым заведениям, группируем по наименованию, считаем количество, сортируем по количеству
top_15_chain = df.query('chain == 1') \
                   .groupby('name') \
                   .agg(count=('name', 'count')) \
                   .sort_values(by='count', ascending=False) \
                   .reset_index() \
                   .head(15)

In [ ]:
#строим график, чтобы посмотреть количество заведений в топ-15 сетях
plt.figure(figsize=(10, 6))
sns.barplot(x='name', 
            y='count', 
            data=top_15_chain) 

plt.title('Топ-15 популярных сетей в Москве')
plt.xlabel('Наименование заведения')
plt.ylabel('Количество заведений', fontsize=12)

plt.xticks(rotation=75)
plt.show()

In [ ]:
#создаем переменную, чтобы посмотреть категории топ-15 сетей в Москве
top_chain_names = top_15_chain['name']

top_15_chain_category = df.query('chain == 1 and name in @top_chain_names') \
                            .groupby(['category', 'name']) \
                            .agg(count=('name', 'count')) \
                            .reset_index() 

In [ ]:
fig = px.bar(top_15_chain_category, 
             x='category', 
             y='count', 
             color='name')
fig.update_layout(title='Количество заведений из популярных сетей по категориям',
                  yaxis_title='Количество заведений',
                  xaxis_title='Категория заведений')
fig.show()

Все топ-15 популярных сетей в Москве мне знакомы, кроме cofefest и хинкальной. Большинство сетей находятся в категории кофейня, лидером среди сетевых заведений в категории кофейня является Шоколадница, при этом заведений Шоколадницы в целом больше всего в Москве - 119.

Додо пицца и домино'с пицца уверенно заполнили рынок пиццерий в Москве: у додо 74 пиццерии, а у домино'с - 76. 

Обратила внимание, что заведения сети Буханка находятся как в категориях кофейня и кафе, так и в категории булочная. Аналогично с хинкальными - они есть сразу в нескольких категориях: ресторан, кафе, бар, паб, быстрое питание. Яндекс Лавка внезапно оказалась в категории ресторанов вместе с Prime. 

#### 12. Изучение общего количества заведений и количества заведений каждой категории по административным районам Москвы <a id="quantity_moscow"></a>

In [ ]:
print('В Москве присутствуют следующие административные районы:', df['district'].unique())

In [ ]:
#в переменной districtcount группируемся по округу и категории, считаем количество заведений 
districtcount = df.groupby(['district', 'category']).agg(count=('name', 'count')).sort_values(by='count', ascending=False).reset_index()
districtcount

In [ ]:
#строим график
fig = px.bar(districtcount.sort_values(by='count', ascending=False), 
             x='district', 
             y='count', 
             color='category')
fig.update_layout(xaxis_title='Район',
                  yaxis_title='Количество заведений')
fig.show()

Больше всего заведений в Центральном административном округе Москвы и большинство из этих заведений - рестораны, на втором месте среди категорий заведений кафе, затем кофейни и чуть меньше баров/пабов. В ЮАО, САО, СВАО примерно одинаковое количество как самих заведений, так и примерно одинаковое распределение этих заведений по категориям. Меньше всего заведений в СЗАО. 

#### 13. Распределение средних рейтингов по категориям заведений <a id="rating"></a>

In [ ]:
rating_mean = pd.pivot_table(df,
              index='category',
              values='rating',
              aggfunc=np.mean).sort_values('rating', ascending=False)
rating_mean

In [ ]:
plt.figure(figsize=(15, 8))
sns.set_style('darkgrid')
sns.boxplot(x='category', y='rating', data=df, hue_order='rating_mean')
plt.title('Распределение средних рейтингов по категориям заведений')
plt.xlabel('Категория')
plt.ylabel('Рейтинг')
plt.show()

Исходя из графика можем сделать следующие выводы: усредненные рейтинги по категориям заведений отличаются не очень сильно, самые высоко оцененные заведения - бары, пабы. Самая низкая средняя оценка у заведений быстрого питания. 

При построении boxplot'а видим следующее: булочные редко оценивают совсем плохо, аналогично с пиццериями. Барами, пабами бывают недовольны, но в целом мы можем увидеть, что оценивают такие места высоко, что не скажешь о кафе, ресторанах и заведениях быстрого питания - там много недовольных, поэтому и средняя оценка ниже. 

#### 14. Построение фоновой картограммы (хороплета) со средним рейтингом заведений каждого района <a id="map"></a>

Границы районов Москвы, которые встречаются в датасете, хранятся в файле admin_level_geomap.geojson.

In [ ]:
#создаем переменную
rating_df = df.groupby('district', as_index=False)['rating'].agg('mean').round(1)
rating_df

In [ ]:
# подключаем модуль для работы с JSON-форматом
import json

# читаем файл и сохраняем в переменной
with open('/datasets/admin_level_geomap.geojson', 'r') as f:
    geo_json = json.load(f)

In [ ]:
# импортируем карту и хороплет
from folium import Map, Choropleth

# загружаем JSON-файл с границами округов Москвы
state_geo = '/datasets/admin_level_geomap.geojson'
# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=rating_df,
    columns=['district', 'rating'],
    key_on='feature.name',
    fill_color='OrRd',
    fill_opacity=0.8,
    legend_name='Средний рейтинг заведений по районам',
).add_to(m)

# выводим карту
m

Самый высокий средний рейтинг в заведениях Центрального административного округа, а вот в ЮВАО и СВАО либо очень требовательные посетители, либо качество заведений ниже - в этих округах средний рейтинг заведений около 4.1.

#### 15. Отображаем все заведения датасета на карте с помощью кластеров средствами библиотеки folium 

In [ ]:
# импортируем карту и маркер
from folium import Map, Marker
# импортируем кластер
from folium.plugins import MarkerCluster

# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(m)

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
df.apply(create_clusters, axis=1)

# выводим карту
m

In [ ]:
print(df.groupby('district')['name'].count().sort_values(ascending=False))

#### 16. Исследование топ-15 улиц по количеству заведений <a id="top_15_streets"></a>

In [ ]:
#создаем переменную top_15_streets, группируемся по улице, считаем количество заведений
top_15_streets = df.groupby('street') \
                   .agg(count=('name', 'count')) \
                   .sort_values(by='count', ascending=False) \
                   .reset_index() \
                   .head(15)
top_15_streets

In [ ]:
top_streets = top_15_streets['street']

top_15_streets_category = df.query('street in @top_streets') \
                            .groupby(['street','category']) \
                            .agg(count=('name', 'count')) \
                            .sort_values(by='count', ascending=False) \
                            .reset_index()

In [ ]:
fig = px.bar(top_15_streets_category, 
             x='street', 
             y='count', 
             color='category').update_xaxes(categoryorder='total descending')
 
fig.update_layout(title='Распределение количества заведений и их категорий по топ-15 улицам Москвы по количеству заведений',
                  yaxis_title='Количество заведений',
                  xaxis_title='Улицы')
fig.show()

Больше всего заведений на проспекте Мира - 183, из них большую часть составляют кафе, рестораны, кофейни. Также много заведений на Профсоюзной улице, проспекте Вернадского, Ленинском проспекте. В целом распределение категорий заведений по топ-15 улицам по количеству заведений одинаковое, за исключением МКАДа - на МКАДе подавляющее большинство заведений - кафе. 

#### 17. Поиск улиц, на которых находится только один объект общепита. 

In [ ]:
#сгруппируем датафрейм по улице и сделаем срез
one_name_street = df.groupby('street') \
                            .agg(count=('name', 'count')) \
                            .sort_values(by='count', ascending=False) \
                            .reset_index() \
                            .query('count == 1')['street']
df_one_name = df.query('street in @one_name_street')

In [ ]:
#через display посмотрим на нашу таблицу
display(df_one_name)

В датасете 458 улиц, на которых только 1 заведение.

In [ ]:
#посмотрим на категории заведений
categories_df_one_name = df_one_name.groupby('category') \
                                   .agg(count=('name', 'count')) \
                                   .sort_values(by='count', ascending=False) \
                                   .reset_index()

In [ ]:
fig = go.Figure(data=[go.Pie(labels=categories_df_one_name['category'], values=categories_df_one_name['count'])])
fig.update_layout(title='Распределение заведений по категориям, которые в единственном числе находятся на улицах Москвы', 
                  width=800, 
                  height=500,
                  annotations=[dict(x=1.12, 
                                    y=1.05,
                                    text='Категории объектов',
                                    showarrow=False)])
fig.show() 

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x='category', 
            y='count', 
            data=categories_df_one_name) 
plt.title('Распределение заведений по категориям, которые в единственном числе находятся на улице в Москве')
plt.xlabel('Категория заведения')
plt.ylabel('Количество заведений', fontsize=12)

plt.xticks(rotation=75)
plt.show()

Заведения, которые находятся в единственном числе на улице, - разных категорий, среди них есть и кафе, и рестораны, и кофейни, и бары. 

In [ ]:
#посмотрим, в каких округах Москвы расположены такие заведения
districts_df_one_name = df_one_name.groupby('district') \
                                   .agg(count=('name', 'count')) \
                                   .sort_values(by='count', ascending=False) \
                                   .reset_index()

In [ ]:
fig = go.Figure(data=[go.Pie(labels=districts_df_one_name['district'], values=districts_df_one_name['count'])])
fig.update_layout(title='Распределение заведений по округам, которые в единственном числе находятся на улице', 
                  width=900, 
                  height=500,
                  annotations=[dict(x=1.12, 
                                    y=1.05,
                                    text='Категории объектов',
                                    showarrow=False)])
fig.show() 

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x='district', 
            y='count', 
            data=districts_df_one_name) 
plt.title('Распределение заведений по округам, которые в единственном числе находятся на улице')
plt.xlabel('Район')
plt.ylabel('Количество заведений', fontsize=12)

plt.xticks(rotation=75)
plt.show()

Один объект общепита часто находится на улицах Центрального административного округа Москвы - 31.7%, также много единственных объектов на улице в Москве в округах СВАО, ВАО и САО - 12%, 11.4% и 11.4% соответственно. 

#### 18. Изучение значений средних чеков заведений по округам Москвы <a id="middle_check"></a>

In [ ]:
#создадим переменную
avg_bill = df.groupby('district')['middle_avg_bill'].median().sort_values()

In [ ]:
display(avg_bill.sort_values(ascending=False))

In [ ]:
# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=avg_bill,
    columns=['district', 'middle_avg_bill'],
    key_on='feature.name',
    fill_color='Purples',
    fill_opacity=0.8,
    legend_name='Средний чек заведений по районам',
).add_to(m)

# выводим карту
m

Предсказуемо самый высокий средний чек в Центральном административном округе Москвы - дорогие заведения и дорогая аренда. Затем идет Западный административный округ - также неудивительно, т.к. на Рублевке и в Раменках очень дорогие цены в заведениях. Средние цены в округах ВАО и САО. А самый низкий средний медианный чек в округах ЮАО, ЮВАО и СВАО.

#### 19. Исследование зависимости рейтинга и среднего чека в заведениях (необязательное задание) <a id="rate_check"></a>

In [ ]:
(
    df.pivot_table(index='rating', values='middle_avg_bill')
    .plot(grid=True, style='o', figsize=(5, 5))
)
plt.show()
print('Корреляция рейтинга заведения и числа с оценкой среднего чека, которое указано только для значений из столбца avg_bill:', df['rating'].corr(df['middle_avg_bill']))

Можно сделать вывод, что размер среднего чека на оценку заведения пользователями не влияет.

**Вывод по результатам проведенного анализа:**
1. Больше всего заведений расположено в Центральном административном округе Москвы, в этом же округе самый высокий средний чек и самый высокий рейтинг заведений.
2. Больше всего в Москве открыто кафе, ресторанов и кофеен.
3. Среднее количество посадочных мест - 108, однако медиана во всех категориях заведений ниже.
4. В Москве сетевые заведения составляют 38.1% всего количества, чаще это заведения категорий булочная, пиццерия, кофейня, несетевых заведений - 61.9%.
5. Усредненные рейтинги по категориям заведений отличаются не очень сильно, самые высоко оцененные заведения - бары, пабы. Самая низкая средняя оценка у заведений быстрого питания. 
6. Больше всего заведений на проспекте Мира - 183, из них большую часть составляют кафе, рестораны, кофейни. Также много заведений на Профсоюзной улице, проспекте Вернадского, Ленинском проспекте. На МКАДе в основном открыты кафе. В Москве целых 458 улиц, на которых всего 1 заведение.

### Детализация исследования: открытие кофейни <a id="final"></a>

Основателям фонда «Shut Up and Take My Money» не даёт покоя успех сериала «Друзья». Их мечта — открыть такую же крутую и доступную, как «Central Perk», кофейню в Москве. Будем считать, что заказчики не боятся конкуренции в этой сфере, ведь кофеен в больших городах уже достаточно. Попробуйте определить, осуществима ли мечта клиентов.

Ответьте на следующие вопросы:
- Сколько всего кофеен в датасете? В каких районах их больше всего, каковы особенности их расположения?
- Есть ли круглосуточные кофейни?
- Какие у кофеен рейтинги? Как они распределяются по районам?
- На какую стоимость чашки капучино стоит ориентироваться при открытии и почему?

По желанию вы можете расширить список вопросов для исследования, добавив собственные.
Постройте визуализации. Попробуйте дать рекомендацию для открытия нового заведения. Это творческое задание: здесь нет правильного или неправильного ответа, но ваше решение должно быть чем-то обосновано. Объяснить свою рекомендацию можно текстом с описанием или маркерами на географической карте.

#### 20. Исследование количества и расположения кофеен в датасете, а также рейтинга кофеен по районам <a id="cafesquantity"></a>

In [ ]:
cafes = df.query('category == "кофейня"')
print('Всего кофеен в датасете:', len(cafes))

16% всех представленных заведений в датасете составляют кофейни - вот это конкуренция.

In [ ]:
# импортируем собственные иконки
from folium.features import CustomIcon

#создаем карту Москвы    
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=cafes,
    columns=['district', 'rating'],
    key_on='feature.name',
    fill_color='OrRd',
    fill_opacity=0.8,
    legend_name='Средний рейтинг кофеен по районам',
).add_to(m)

# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(m)

def create_clusters(row):
    # сохраняем URL-адрес изображения со значком кафе с icons8,
    # это путь к файлу на сервере icons8
    icon_url = 'https://img.icons8.com/?size=512&id=YqniqtUYqD68&format=png'  
    # создаём объект с собственной иконкой размером 30x30
    icon = CustomIcon(icon_url, icon_size=(30, 30))

# создаём маркер с иконкой icon и добавляем его в кластер
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
        icon=icon,
    ).add_to(marker_cluster)

# применяем функцию для создания кластеров к каждой строке датафрейма
cafes.apply(create_clusters, axis=1)

# выводим карту
m

In [ ]:
#посмотрим на количество кофеен в округах
print(cafes.groupby('district')['name'].count().sort_values(ascending=False))

In [ ]:
#сделаем расчет по рейтингам
rating_cafes = cafes.groupby('district', as_index=False)['rating'].agg('mean').round(1).sort_values(by='rating', ascending=False)
rating_cafes

Построив хороплет видим, что большинство кофеен находится в Центральном административном округе - очень много кофеен в Замоскворечье, а также в самом центре - вокруг Кремля и Красной площади. Чуть меньше кофеен находится в Северном, Северо-Западном, Южном и Восточном округах. 

Кофейни часто находятся в торговых центрах или расположены в жилых комплексах.

При этом в ЦАО у кофеен в основном низкий рейтинг - 4.3. - 4.4. (при этом ранее мы установили, что средний рейтинг всех категорий заведений в ЦАО самый высокий), а вот в СВАО и ВАО у кофеен очень высокие рейтинги.

#### 21. Исследование наличия круглосуточных кофеен <a id="cafes24_7"></a>

In [ ]:
cafes_24_7 = cafes.query('hours == "ежедневно, круглосуточно"')
print('Всего круглосуточных кофеен в датасете:', len(cafes_24_7))

#### 22. Исследование стоимости чашки капучино в кофейнях по районам <a id="cap_sum"></a>

In [ ]:
coffee_cup = cafes.query('middle_coffee_cup != 0') \
                                    .groupby('district') \
                                    .agg(mean_middle_coffee_cup=('middle_coffee_cup', 'mean')) \
                                    .sort_values(by='mean_middle_coffee_cup', ascending=False) \
                                    .reset_index()

In [ ]:
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

Choropleth(
    geo_data=state_geo,
    data=coffee_cup,
    columns=['district', 'mean_middle_coffee_cup'],
    key_on='feature.name',
    fill_opacity=0.8,
    legend_name='Стоимость чашки капучино по округам',
).add_to(m)

m

#### 23. Исследование количества посадочных мест в кофейнях (необязательное задание)

In [ ]:
plt.figure(figsize=(15, 8))
plt.hist(cafes['seats'], bins=100)
plt.title('Количество посадочных мест в кофейнях')
plt.xlabel('Количество посадочных мест')
plt.ylabel('Количество значений')
plt.show()

**По результатам исследовательского анализа и детализации исследования хочу дать следующие рекомендации для открытия кофейни:**
1. Рекомендуется открывать кофейню в Восточном административном округе Москвы: в этом округе цена чашки капучино выше среднего, а это значит, что инвестиции в открытие отобьются быстрее. 
2. В этом округе конкуренция меньше, чем в 5 других округах (заведений меньше).
3. Средний рейтинг кофеен в этом округе выше, чем в остальных. 
4. В ВАО много новостроек и много зеленых зон, соответственно, много семей с детьми, много велосипедистов, много людей с собаками, соответственно, будет большая проходимость. 

Ниже указала с помощью "капли" 2 рекомендуемых места для открытия кофейни:
1. Первое с координатами: 37.701956,55.830013 на входе в Лосиный остров. Во-первых, на входе в парк нет ни одного заведения. Во-вторых, рядом 2 здания институтов: МГГЭУ и РГСУ, а значит, что будут ходить студенты. Плюс рядом район Метрогородка, где много домов, а заведений общественного питания также нет. Заведение сразу на выходе с МЦК. Ну и рядом парк "Друзья леса" - все сходится.
2. Второе с координатами: 55.826347, 37.571022. Это точка не в ВАО, а в САО - на границе со СВАО. Рядом нет кофеен, при этом по близости есть метро, несколько парков и большой институт. 

Кофейни на 80 посадочных мест (медиана по кофейням) будет достаточно. 

In [ ]:
# подключаем библиотеку
import folium

# сохраняем координаты рекомендуемого места (ВАО) для открытия кофейни в переменные
friends_cafe_lat, friends_cafe_lng = 55.830617, 37.702896
# сохраняем координаты рекомендуемого места (САО) для открытия кофейни в переменные
friends_cafe_2_lat, friends_cafe_2_lng = 55.826347, 37.571022

# создаём карту с центром в точке расположения рекомендуемого мной места и начальным зумом 17
m = folium.Map(location=[friends_cafe_lat, friends_cafe_lng], zoom_start=17)
# создаём маркер в точке расположения Большого театра и сразу добавляем на карту
folium.Marker([friends_cafe_lat, friends_cafe_lng]).add_to(m)
folium.Marker([friends_cafe_2_lat, friends_cafe_2_lng]).add_to(m)

# выводим карту
m

Ссылка на презентацию: https://disk.yandex.ru/i/SXRXzTDX22QXRw